In [4]:
import os
import pandas as pd
import plotly.express as px

# Get the current working directory
current_path = os.getcwd()
print("Current Path:", current_path)

# Define the input file path using the current directory
input_file_path = os.path.join(current_path, "inputs/modified_energy_cap.csv")
modified_energy_data = pd.read_csv(input_file_path)

# Function to normalize values to a 0-1 scale
def normalize(x):
    return (x - x.min()) / (x.max() - x.min())

# Apply normalization to the 'energy_cap' within each 'names' group
modified_energy_data['scaled_energy_cap'] = modified_energy_data.groupby('names')['energy_cap'].transform(normalize)


# Create the scatter plot
fig = px.scatter(modified_energy_data, x='names', y='scaled_energy_cap',
                 color='colors', symbol='spores_numeric',
                 labels={
                     'names': 'Technology Names',
                     'energy_cap': 'Energy Capacity (MW)'
                 },
                 title='Energy Capacity by Technology Names Across Different Spores')

# Show the figure
fig.show()
    

Current Path: c:\Users\wangn\OneDrive - TNO\Repositories\mgaPlots


In [5]:
import os
import pandas as pd
import plotly.graph_objects as go

# Get the current working directory
current_path = os.getcwd()
print("Current Path:", current_path)

# Define the input file path using the current directory
input_file_path = os.path.join(current_path, "inputs/modified_energy_cap.csv")
modified_energy_data = pd.read_csv(input_file_path)

# Function to normalize values to a 0-1 scale
def normalize(x):
    return (x - x.min()) / (x.max() - x.min())

# Apply normalization to the 'energy_cap' within each 'names' group
modified_energy_data['scaled_energy_cap'] = modified_energy_data.groupby('names')['energy_cap'].transform(normalize)


# Create the plot
fig = go.Figure()

# Get unique spore_numeric values
spore_numbers = modified_energy_data['spores_numeric'].unique()

# Adding scatter points for each spore_numeric
for spore in spore_numbers:
    spore_data = modified_energy_data[modified_energy_data['spores_numeric'] == spore]
    # Add scatter trace for points
    fig.add_trace(go.Scatter(
        x=spore_data['names'], y=spore_data['scaled_energy_cap'],
        mode='markers', name=f'Spore {spore}',
        marker=dict(color=spore_data['colors'], size=10, line=dict(width=1)),
        hoverinfo='text',
        text=spore_data['names'] + '<br>Cap: ' + spore_data['scaled_energy_cap'].astype(str),
        legendgroup=f'spore_{spore}'
    ))
    # Add line trace for connecting points, with reduced opacity
    fig.add_trace(go.Scatter(
        x=spore_data['names'], y=spore_data['scaled_energy_cap'],
        mode='lines', name=f'Line Spore {spore}',
        line=dict(color='grey', width=2, dash='dot'),
        opacity=0.2,  # Reduced opacity makes the line faint
        hoverinfo='skip',  # Avoid hover for lines to keep the chart clean
        showlegend=False,
        legendgroup=f'spore_{spore}'
    ))

# Update layout to improve readability
fig.update_layout(
    title='Energy Capacity by Technology Names Across Different Spores',
    xaxis_title='Technology Names',
    yaxis_title='Scaled Energy Capacity',
    hovermode='closest'  # This ensures hover effects are visible closely to the cursor
)

# Show the figure
fig.show()


Current Path: c:\Users\wangn\OneDrive - TNO\Repositories\mgaPlots


In [6]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Load your data
df = modified_energy_data

# Normalize 'energy_cap' within each 'names' group
df['scaled_energy_cap'] = df.groupby('names')['energy_cap'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

app = Dash(__name__)

# Create a figure and update it in the callback
@app.callback(
    Output('graph', 'figure'),
    [Input('graph', 'hoverData')],
    [State('graph', 'figure')])
def update_highlight(hoverData, fig):
    if hoverData:
        spore_num = hoverData['points'][0]['customdata']
        # Set opacity low for all points and higher for points with the same spore_numeric
        for trace in fig['data']:
            if trace['customdata'][0] == spore_num:
                trace['marker']['opacity'] = 1
                trace['marker']['size'] = 20
                trace['textfont'] = dict(size=14)  # Show labels with larger text size
            else:
                trace['marker']['opacity'] = 0.01
                trace['marker']['size'] = 12
                trace['textfont'] = dict(size=0)
    else:
        # Reset opacities and marker sizes to initial state when not hovering
        for trace in fig['data']:
            trace['marker']['opacity'] = 1
            trace['marker']['size'] = 12
            trace['textfont'] = dict(size=0)  # Hide labels by default
    return fig

# Initial Figure
fig = go.Figure(data=[
    go.Scatter(
        x=df[df['spores_numeric'] == num]['names'],
        y=df[df['spores_numeric'] == num]['scaled_energy_cap'],
        mode='markers',
        customdata=[num]*len(df[df['spores_numeric'] == num]),
        marker={'size': 12},
        name=f'Spore {num}'
    ) for num in df['spores_numeric'].unique()
])

# Save the figure as an HTML file
html_file_path = os.path.join(current_path, "Energy_Capacity_Plot.html")
fig.write_html(html_file_path)

app.layout = html.Div([
    dcc.Graph(id='graph', figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
